In [66]:
import pandas as pd

# Movie Revenue Predictions

Drie concept-onderzoeksvragen:
- In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?
- 2
- 3

## Het Data Science proces
Voor de eerste verkenning is ons gevraagd om de eerste vier stappen uit te voeren:
1. Data collection
2. Data processing (ook wel data munging)
3. Data cleaning
4. Data exploration & analysis
5. Model building
6. Visualization
6. Communication


## 1. Data Collection
De Data Collection is deels al voor ons gedaan. De dataset `movie.csv` is ons aangeleverd. Echter word voor de opdracht 
gevraagd om deze te combineren met een dataset van derden. Om erachter te komen welke dataset geschikt is om te 
combineren met `movies.csv`zullen wij deze dataset eerst moeten processen, cleanen en exploren.  

Om te zien of de dataset `movie.csv` goed is ingeladen, worden de eerste vijf rijen getoond:

In [67]:
df_movies = pd.read_csv('data/movie.csv')
df_movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


## 2. Data Processing
Ook deze stap is grotendeels voor ons gedaan. De data is goed opgeslagen in een `.csv`-bestand en kan direct worden 
ingelezen in een _Pandas_ DataFrame.

Verder rest ons nog de volgende drie stappen:
1. Ongewenste kolommen droppen
2. Onduidelijke kolomnamen veranderen
3. Volgorde kolommen aanpassen
4. Datatypes aanpassen

In [68]:
df_movies.drop(["movie_imdb_link", "aspect_ratio"], axis=1, inplace=True)

df_movies.rename(columns={'color':'Colour',
'director_name': 'Director',
'num_critic_for_reviews': 'Number of critics',
'duration': 'Duration',
'director_facebook_likes': 'Director FB likes',
'actor_3_facebook_likes': 'Actor 3 FB likes',
'actor_2_name': 'Actor 2 name',
'actor_1_facebook_likes': 'Actor 1 FB likes',
'gross': 'Gross',
'genres': 'Genres',
'actor_1_name': 'Actor 1 name',
'movie_title': 'Movie title',
'num_voted_users': 'Number of voted users',
'cast_total_facebook_likes': 'Total Cast FB likes',
'actor_3_name': 'Actor 3 name',
'facenumber_in_poster': 'Number of faces on poster',
'plot_keywords': 'Plot Keywords',
'num_user_for_reviews': 'Number of user reviews',
'language': 'Language',
'country': 'Country',
'content_rating': 'Age rating',
'budget': 'Budget',
'title_year': 'Release year',
'actor_2_facebook_likes': 'Actor 2 FB likes',
'imdb_score': 'IMDB Score',
'movie_facebook_likes': 'Movie FB likes'}, inplace=True)

# Volgorde kolommen aanpassen

df_movies["Release year"] = pd.to_datetime(df_movies["Release year"], format='%Y')

De DataFrame ziet er na stap 2. Data Processing als volgt uit:

In [69]:
df_movies.head()

,Colour,Director,Number of critics,Duration,Director FB likes,Actor 3 FB likes,Actor 2 name,Actor 1 FB likes,Gross,Genres,...,Plot Keywords,Number of user reviews,Language,Country,Age rating,Budget,Release year,Actor 2 FB likes,IMDB Score,Movie FB likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,avatar|future|marine|native|paraplegic,3054.0,English,USA,PG-13,237000000.0,2009-01-01,936.0,7.9,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,goddess|marriage ceremony|marriage proposal|pi...,1238.0,English,USA,PG-13,300000000.0,2007-01-01,5000.0,7.1,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,bomb|espionage|sequel|spy|terrorist,994.0,English,UK,PG-13,245000000.0,2015-01-01,393.0,6.8,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,deception|imprisonment|lawlessness|police offi...,2701.0,English,USA,PG-13,250000000.0,2012-01-01,23000.0,8.5,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,12.0,7.1,0


## 3. Data Cleaning
1. NaN-types verwijderen
2. Dubbele Movie Titles verwijderen

In [70]:
df_movies.dropna(inplace=True)

Na stap 3. Data Cleaning ziet het DataFrame er als volgt uit:

In [71]:
df_movies.head()

,Colour,Director,Number of critics,Duration,Director FB likes,Actor 3 FB likes,Actor 2 name,Actor 1 FB likes,Gross,Genres,...,Plot Keywords,Number of user reviews,Language,Country,Age rating,Budget,Release year,Actor 2 FB likes,IMDB Score,Movie FB likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,avatar|future|marine|native|paraplegic,3054.0,English,USA,PG-13,237000000.0,2009-01-01,936.0,7.9,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,goddess|marriage ceremony|marriage proposal|pi...,1238.0,English,USA,PG-13,300000000.0,2007-01-01,5000.0,7.1,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,bomb|espionage|sequel|spy|terrorist,994.0,English,UK,PG-13,245000000.0,2015-01-01,393.0,6.8,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,deception|imprisonment|lawlessness|police offi...,2701.0,English,USA,PG-13,250000000.0,2012-01-01,23000.0,8.5,164000
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,alien|american civil war|male nipple|mars|prin...,738.0,English,USA,PG-13,263700000.0,2012-01-01,632.0,6.6,24000


## 4. Data Exploration & Analysis

In [57]:
# df_movies.describe(include = "all")

In [58]:
# df_movies.count()

In [59]:
# df_movies.dtypes

In [51]:
# df_movies.set_index("Movie title", inplace=True)
# df_movies.sort_index(inplace=True)